### Teste etapa 1

In [2]:
from tree import Tree
from utils import inputs

In [3]:
expression_trees = map(Tree.from_expression, inputs)

for idx, tree in enumerate(expression_trees):
    print(f'Arvore {idx+1} :')
    tree.bshow()
    print()

Arvore 1 :
┌───
MEM
│   ┌───CONST 2
└───+
    └───CONST 1

Arvore 2 :
┌───
MOVE
│       ┌───
│   ┌───MEM
│   │   │       ┌───
│   │   │   ┌───MEM
│   │   │   │   │   ┌───CONST a
│   │   │   │   └───+
│   │   │   │       └───FP
│   │   └───+
│   │       │   ┌───CONST 4
│   │       └───*
│   │           └───TEMP i
└───MEM
    │   ┌───CONST x
    └───+
        └───FP



### Fase de seleção de instrução

In [ ]:


JOUETTE_ARCHITECTURE = {
    **{
        ins: {
            'patterns': [Tree(op)],
            'instruction': f'ri <- rj {op} rk'
        }  
        for ins, op in [('ADD', '+'), ('MUL', '*'), ('SUB', '-'), ('DIV', '/')]
    },
    'ADDI': {
        'patterns': [
            Tree('+', right='CONST'),
            Tree('+', left='CONST'),
            Tree('CONST')
        ],
        'instruction': 'ri <- rj + c' 
    }

}